In [1]:
%cd /aiffel

from copy import deepcopy
import json

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

/aiffel


In [2]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [3]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [4]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [5]:
with open('aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [6]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [7]:
len(list_prompt)

12000

In [8]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [9]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)



Train epoch [1/1]: 100%|██████████| 3/3 [00:01<00:00,  1.84it/s, actor_loss=-.00919, critic_loss=0.00039]


Episode [10/10]: 100%|██████████| 3/3 [00:19<00:00,  6.49s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3 │   │   │   max_timesteps=3,                                                                 │
│   4 │   │   │   update_timesteps=3)                                                              │
│   5                                                                                              │
│ ❱ 6 model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')                                       │
│   7                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [12]:
actor.model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')

In [13]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'죄송합니다, 저는 인공지능 언어모델로 생성된 어시스턴트이므로 쇠고기용 쇠고기 한우의 종류를 말씀하셔야합니다. 따라서, 해당 가게의 정보를 확인하시기 바랍니다. 해당 가게의 웹사이트를 참고하시기 바랍니다.\n\n저는 인공지능 챗봇으로서 사료, 약어, 식용 및 유통 상황에 대해 직접 답변하고 있으며 이 질문이 어떤 상황에서 어떤 방식으로든 중요합니다. 이에 대한 정보가 있으면 더 자세한 답변을 드릴 수 있습니다.賤賤賤)으로 불리기도 하실 것 같습니다.慎宗永)에 의하면, "불고기" 라고 부릅니다.晨宗宗永)에 따르면, "불고기"로 불리기도 하실 것으로 추정됩니다.臣宗永에 따르면, "불고기" 라고 부릅니다.臣上宗永은 일반적으로 "불고기" 이라는 의미로 사용되는 것으로 알려져 있습니다.、神上宗永은 일반적으로 '불고기" 라고 부릅니다.神常宗永은 "불고기" 라고 부릅니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 33대 부통령직을 수행한 년도에 대해 발표하지 않았습니다. \'리처드 닉슨은 46대 부통령직이 수행된 년도에 대한 정보가 없으므로 정확한 답변을 제공할 수 없습니다. \'리처드 닉슨은 46대 부통령직을 수행한 년도에 대해서도 발표하지 않았습니다. \'리처드 닉슨은 42대 부통령직을 수행한 년도에 대해서는 정보가 없습니다. \'리처드 닉슨은 41대 부통령직을 수행한 년도에 대한 정보 없습니다. \'리처드 닉슨은 42대 부통령직을 수행한 년도에 대한 정보가 제공되지 않는 것 같습니다. \'리처드 닉슨은 44대 부통령직을 수행한 년도에 대한 정보가 없습니다. \'리처드 닉슨은 46대 부통령직을 수행한 년도에 대한 정보가 존재하지 않습니다. \'리처드 닉슨은 43대 부통령직을 수행한 년도에 대해 언급하지 않습니다. \'리처드 닉슨은 42대 부통령직을 수행한 년도에 대한 정보는 없